In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from datasets import Dataset

## Вставляй свои пути!!!

In [ ]:
train = pd.read_csv('train (19).csv') # !!!
test = pd.read_csv('test (24).csv') # !!!
ss = pd.read_csv('sample_submission (26).csv') # !!!

## Выбирай тут модель (начинай с deep_pavlov base, переходи у более крупным)

In [ ]:
model_name = 'DeepPavlov/rubert-base-cased'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name,  num_labels=2)

In [ ]:
from datasets import Dataset

train_df = Dataset.from_pandas(pd.DataFrame({'text': train['text'] , 'labels': train['target']}))
# СВОЕ ВСТАВЛЯЙЙ!!!!
test_df = Dataset.from_pandas(pd.DataFrame({'text': test['text']}))
# аналогично


In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, padding='longest')

In [ ]:
train_df_t = train_df.map(tokenize, batched=True)

test_df_t = test_df.map(tokenize, batched=True)

In [ ]:
train_args = TrainingArguments(
    output_dir='idk',
    num_train_epochs=3, # если A100 фигачит быстро, то поставь побольше хз
    per_device_train_batch_size=16, # и тут аналогичноо
    learning_rate=3e-5,
    logging_steps=500,
    save_strategy="epoch",
    logging_dir="idk",
    report_to="none",
    bf16=True, #ибо A100б
    warmup_ratio=0.1,
    weight_decay=0.01

)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=-1)
    return {'accuracy': accuracy_score(labels, preds)}


In [ ]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_df_t,
    compute_metrics=compute_metrics
)
trainer.train()

In [ ]:
preds = trainer.predict(test_df_t)
y_pred = np.argmax(preds.predictions, axis=-1)

# разные уровни очистки, лучше просто оставь 1 уровень

In [ ]:
import re
import string
from typing import List

# -----------------------------
# LEVEL 0: минимальная обработка
# -----------------------------
def preprocess_level_0(text: str) -> str:
    """
    Удаляем только невидимые символы и лишние пробелы.
    Это полностью безопасно для RuBERT.
    """
    text = text.replace("\n", " ").replace("\t", " ")
    text = re.sub(r"\s+", " ", text).strip()
    return text


# -----------------------------
# LEVEL 1: лёгкая очистка
# -----------------------------
def preprocess_level_1(text: str) -> str:
    """
    Уровень 0 + удаление URL, упрощение повторов пробелов.
    """
    text = preprocess_level_0(text)
    
    # Удаление ссылок
    text = re.sub(r"http\S+|www\.\S+", "", text)
    
    # Удаление мусорных спецсимволов
    text = re.sub(r"[«»¬]", "", text)
    return text


# -----------------------------
# LEVEL 2: умеренная очистка
# -----------------------------
def preprocess_level_2(text: str) -> str:
    """
    Уровень 1 + удаление эмодзи + трансформация повторов знаков.
    """
    text = preprocess_level_1(text)

    # Удаление emoji
    emoji_pattern = re.compile(
        "["  
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub("", text)

    # Сведение !!! или ??? к одному знаку
    text = re.sub(r"!{2,}", "!", text)
    text = re.sub(r"\?{2,}", "?", text)

    return text


# -----------------------------
# LEVEL 3: глубокая очистка
# -----------------------------
def preprocess_level_3(text: str) -> str:
    """
    Уровень 2 + удаление пунктуации, кроме важных знаков.
    """
    text = preprocess_level_2(text)

    # Удаляем всю пунктуацию, оставляем только . ! ?
    allowed = ".!?"
    text = "".join(ch for ch in text if ch.isalnum() or ch.isspace() or ch in allowed)

    # Удаление 2+ пробелов
    text = re.sub(r"\s+", " ", text).strip()

    return text


# -----------------------------
# LEVEL 4: агрессивная нормализация
# -----------------------------
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def preprocess_level_4(text: str) -> str:
    """
    Уровень 3 + приведение к нижнему регистру + лемматизация.
    ⚠ Может ухудшать качество BERT (cased модель!).
    """
    text = preprocess_level_3(text)

    # Нижний регистр
    text = text.lower()

    # Лемматизация
    tokens = text.split()
    lemmas = []
    for token in tokens:
        parsed = morph.parse(token)
        if parsed:
            lemmas.append(parsed[0].normal_form)
        else:
            lemmas.append(token)

    return " ".join(lemmas)


# а лучше вообще вот это

In [ ]:
def clean_text(text):
    """Очистка текста"""
    if not isinstance(text, str):
        return ""
            
    # Приведение к нижнему регистру
    text = text.lower()
        
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()
        
        # Удаление URL
    text = re.sub(r'http\S+', '', text)
        
        # Удаление email
    text = re.sub(r'\S+@\S+', '', text)
        
        # Удаление лишних знаков препинания (можно настроить)
    text = re.sub(r'[^\w\s]', ' ', text)
        
    return text

In [ ]:
import re

def preprocess_medium(text: str) -> str:                         # убрать пробелы по краям
    text = re.sub(r'\s+', ' ', text)              # один пробел вместо множества
    text = re.sub(r'[^\S\r\n]+', ' ', text)       # убрать "странные" пробелы

    text = text.lower()                           # привести к нижнему регистру
    text = re.sub(r'https?://\S+', '', text)      # удалить ссылки
    text = re.sub(r'@\w+', '', text)              # удалить упоминания @user
    text = re.sub(r'#\w+', '', text)              # удалить хештеги
    text = re.sub(r'[^\w\s,.!?—-]', ' ', text)    # убрать всё, кроме рус/англ букв и пунктуации
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


## вот полный код для копирования

In [ ]:
import pandas as pd
import numpy as np
import re  # Добавь эту строку!
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from datasets import Dataset
from sklearn.metrics import accuracy_score

train = pd.read_csv('train (19).csv') # !!!
test = pd.read_csv('test (24).csv') # !!!
ss = pd.read_csv('sample_submission (26).csv') # !!!

model_name = 'DeepPavlov/rubert-base-cased'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,  num_labels=2)

def clean_text(text):
    """Очистка текста"""
    if not isinstance(text, str):
        return ""
            
    # Приведение к нижнему регистру
    text = text.lower()
        
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()
        
    # Удаление URL
    text = re.sub(r'http\S+', '', text)
        
    # Удаление email
    text = re.sub(r'\S+@\S+', '', text)
        
    # Удаление лишних знаков препинания (можно настроить)
    text = re.sub(r'[^\w\s]', ' ', text)
        
    return text

# ОЧИСТКА ТЕКСТА - ДОБАВЬ ЭТО!
train['text'] = [clean_text(text) for text in train['text']]
test['text'] = [clean_text(text) for text in test['text']]

train_df = Dataset.from_pandas(pd.DataFrame({'text': train['text'], 'labels': train['target']}))
test_df = Dataset.from_pandas(pd.DataFrame({'text': test['text']}))

def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, padding='longest')

train_df_t = train_df.map(tokenize, batched=True)
test_df_t = test_df.map(tokenize, batched=True)

train_args = TrainingArguments(
    output_dir='idk',
    num_train_epochs=3, # если A100 фигачит быстро, то поставь побольше хз
    per_device_train_batch_size=16, # и тут аналогичноо
    learning_rate=3e-5,
    logging_steps=500,
    save_strategy="epoch",
    logging_dir="idk",
    report_to="none",
    bf16=True, #ибо A100
    warmup_ratio=0.1,
    weight_decay=0.01
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=-1)
    return {'accuracy': accuracy_score(labels, preds)}

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_df_t,
    compute_metrics=compute_metrics
)
trainer.train()

preds = trainer.predict(test_df_t)
y_pred = np.argmax(preds.predictions, axis=-1)



In [ ]:
def clean_text_basic(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text


# Это брать эмбеддинги

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Шаг 1: Загрузка модели и токенизатора
model_name = "DeepPavlov/rubert-base-cased"  # или "DeepPavlov/rubert-base-cased" для русского
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Шаг 2: Подготовка текста
text = "Hello, how are you today?"
# или для нескольких текстов:
texts = ["First sentence", "Second sentence here"]

# Шаг 3: Токенизация
inputs = tokenizer(
    text, 
    return_tensors="pt",           # Возвращать PyTorch tensors
    padding=True,                  # Дополнять до одинаковой длины
    truncation=True,               # Обрезать слишком длинные последовательности
    max_length=512,                # Максимальная длина
    add_special_tokens=True        # Добавлять [CLS] и [SEP]
)

print("Токенизированный вход:")
print(f"input_ids: {inputs['input_ids']}")
print(f"attention_mask: {inputs['attention_mask']}")

# Переводим модель в режим оценки (важно для consistency)
model.eval()

# Отключаем вычисление градиентов для экономии памяти
with torch.no_grad():
    outputs = model(**inputs)

# last_hidden_state - это то, что нам нужно!
last_hidden_state = outputs.last_hidden_state

print(f"\nФорма last_hidden_state: {last_hidden_state.shape}")
# Для одного предложения: [batch_size, sequence_length, hidden_size]
# Пример: torch.Size([1, 9, 768])

1047473923787078544500265828987198637094183068158903134027400531278227968366832713890395951478238578909080077778834750879346196878338281787942580947821121009003323721104499718179177712541036988621951891182890900009588283186165606410536493610985560606324763572675049845098756673532884720865719117383551588096767527591934189855262026069986528290668415574475496345472963546345615318715492659204223113867495830369852772997711500683649717661259743896537321393367836005101460391220832801063226365850174163265167827097140668101992555584226337973004457008875414264468076040095244899944697223266667712847552732886340928819636625781312469736087751951431828780682313864349370772509918310474330264539506965501095279506552561442463615338250957486354972366501399614873280757364529882726213983121887216197698634353398387566210983369466251960045092867067316358291890771945137503303161710447059217809626923653431685160925744000265042450640952159985110959999014892471579297063056840467871515200035322760337702379463419

# Усреднение векторов для CatBoost

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

def get_sentence_embedding(texts, model_name='DeepPavlov/rubert-base-cased'):
    """Получает эмбеддинги предложений через усреднение токенов"""
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    
    # Токенизация
    inputs = tokenizer(
        texts, 
        return_tensors="pt", 
        padding=True, 
        truncation=True, 
        max_length=512
    )
    
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
    
    last_hidden_state = outputs.last_hidden_state
    attention_mask = inputs['attention_mask']
    
    # Усреднение с учетом маски
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
    sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    mean_embeddings = sum_embeddings / sum_mask
    
    return mean_embeddings.numpy()


texts = ["This is first text", "Another text for classification"]
embeddings = get_sentence_embedding(texts)

print(f"Форма эмбеддингов для CatBoost: {embeddings.shape}")
# 

In [ ]:
# pip install sentence-transformers
from sentence_transformers import SentenceTransformer

def get_sbert_embeddings(texts, model_name='all-MiniLM-L6-v2'):
    """Использует специализированные модели для эмбеддингов"""
    model = SentenceTransformer(model_name)
    embeddings = model.encode(texts)
    return embeddings

# Использование
sbert_embeddings = get_sbert_embeddings(texts)
print(f"Форма SBERT эмбеддингов: {sbert_embeddings.shape}")

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

def get_embeddings(texts):
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    model = AutoModel.from_pretrained('bert-base-uncased')
    
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Усредняем эмбеддинги токенов
    mask = inputs['attention_mask'].unsqueeze(-1)
    embeddings = (outputs.last_hidden_state * mask).sum(1) / mask.sum(1)
    
    return embeddings.numpy()

# Использование
X = get_embeddings(your_texts)
# Теперь X готов для CatBoost!